# CNTK Test Platform
Can compare the results of running onnx model on the CNTK and running hdf5 model on Keras.

In [ ]:
import sys
sys.path.insert(0, "../onnx-keras/")
import frontend

from keras.models import load_model
import keras.layers as Klayers
from keras.models import Model
import numpy as np

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("Ensure debug outputs")

In [ ]:
np.set_printoptions(suppress=True, formatter={'float': '{:0.6f}'.format})

## Initialization
Set the path of models and the input size. Load the model. Or construct a model yourself.

In [ ]:
# Configurations
# KerasFile = "/home/jiyuan/Work/playground/submodel1_0_256_0.97_0.92_270.hdf5"
# OnnxFile = "/home/jiyuan/Work/playground/model.onnx"
# KerasFile = "C:\\Users\\Joel Liu\\Work\\kneron_gesnet.hdf5"
KerasFile = "C:\\Users\\Joel Liu\\Work\\tiny-yolo-voc.hdf5"
OnnxFile = "C:\\Users\\Joel Liu\\Work\\model.onnx"

In [ ]:
# Reload the frontend
import importlib
importlib.reload(frontend)
rand_input = np.random.rand(1, 224, 224, 3).astype('float32')

In [ ]:
# Load from file
k_model = load_model(KerasFile)
# layer_name = 'batch_normalization_1'
# k_model = Model(inputs=k_model.input, outputs=k_model.get_layer(layer_name).output)

In [ ]:
#Construct testing model
# inputs = Klayers.Input(shape=(112,))
# x = Klayers.Reshape((1, 1, 112))(inputs)

# k_model = Model(inputs=inputs, outputs=x)
# k_model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # a = np.random.rand(1, 32).astype('float32')
# # b = np.random.rand(1, 32).astype('float32')
# # k_model.fit(x=a,y=b)

# k_model.summary()

## Conversion
Convert the model into onnx.

In [ ]:
# Onnx part
converter = frontend.KerasFrontend()
converter.loadFromModel(k_model)
onnx_model = converter.convertToOnnx()
converter.saveToFile(OnnxFile)

## Comparison
The section below will compare two results. However, there are still bugs in CNTK. So wrong results does not indicate that the onnx model is wrong. Here list the known bugs in CNTK.

1. CNTK cannot deal with epsilon other than 1e-5 in the BatchNormalization layer.
2. CNTK cannot deal with Reshape layers.

In [ ]:
#　Keras part
k_out = k_model.predict(rand_input)

In [ ]:
# pre = k_out

In [ ]:
# CNTK part
import cntk as C

z = C.Function.load(OnnxFile, device=C.device.cpu(), format=C.ModelFormat.ONNX)
z_out = z.eval(np.transpose(rand_input, [0, 3, 1, 2]))

In [ ]:
# Compare
print(converter.ops)
if len(k_out.shape) == 4:
    k_out_t = np.transpose(k_out, [0, 3, 1, 2])
else:
    k_out_t = k_out
# if len(z_out.shape) != len(k_out.shape):
#     z_out = z_out.reshape(np.shape(k_out.shape))
result = z_out - k_out_t
abs_result = np.absolute(result)
threshold = 0.01

if (abs_result > threshold).any():
    print("Wrong values (threshold: {}):".format(threshold))
    wv = abs_result[np.where(abs_result > threshold)]
    print(sorted(wv, reverse=True)[:10])
    print("Count: {} - {:.2f}%".format(len(wv), len(wv) / abs_result.size * 100))
#     print("Difference:")
#     print(result)
else:
    print("Test passed (threshold: {})".format(threshold))
# print("Input:")
# print(np.transpose(pre, [0, 3, 1, 2]))
# print("Keras output:")
# print(k_out_t)
# print("CNTK output:")
# print(z_out)

In [ ]:
# k_model.get_layer('batch_normalization_1').get_weights()

In [ ]:
# (-0.549246 - -0.291988)/(0.036777 + 1e-3)**0.5 * 2.524630 + -1.684543

In [ ]:
# with open('C:\\Users\\Joel Liu\\Work\\model.txt', 'w') as f:
#     f.write(str(onnx_model))
#     f.close()
    
# print(onnx_model)